In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import math
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='CRITICAL')

import os
import sys
from pathlib import Path
import IPython.display as ipd
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
from PySpice.Plot.BodeDiagram import bode_diagram
from PySpice.Plot.BodeDiagram import bode_diagram_gain

import schemdraw
import schemdraw.elements as elm
from schemdraw import logic
from schemdraw import dsp

directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)
## set the project directory as directory_path
directory_path = Path(os.path.abspath('')).resolve()

import json
from IPython.display import display, HTML
from IPython.display import Javascript

display(HTML('''
<script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/3.5.0/chart.min.js" integrity="sha512-asxKqQghC1oBShyhiBwA+YgotaSYKxGP1rcSYTDrB0U6DxwlJjU59B67U8+5/++uFjcuVM8Hh5cokLjZlhm3Vg==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>
'''))

In [ ]:
class VoltageDivider(SubCircuitFactory):
    __name__ = 'voltage_divider'
    __nodes__ = ('n1', 'n2', 'n3' )
    __R = 100@u_kΩ

    def __init__(self, R=100@u_kΩ, w=0.4, name='voltage_divider'):
        self.__R__ = R
        SubCircuit.__init__(self, name, *self.__nodes__)
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )
        
    def wiper(self, w) :
        if w == 0 :
            self.R1.resistance = self.__R * 0.0000001
            self.R2.resistance = self.__R * 0.9999999
        elif w == 1 :
            self.R1.resistance = self.__R * 0.9999999
            self.R2.resistance = self.__R * 0.0000001
        else :
            self.R1.resistance = self.__R * w
            self.R2.resistance = self.__R * (1.0-w)


# basic envelope

There are different types of envelope generators. you see a lot of implementations with logic chips or even the 555 timer. It is the goal to build a envelope generator with opamps. A very basic opamp attack release envelope can look like this.

In [ ]:
V1, V2, R1, R2 = sp.symbols("V1 V2 R1 R2")
eq1 = sp.Eq(V2, V1 * R2 / (R1 + R2))

eq1_R1 = sp.solve(eq1, R1)[0]
print(eq1_R1)
f_result_R1 = eq1_R1.evalf(subs={R2: 10@u_kOhm, V1: 15@u_V, V2: 2.5@u_V})
print(f_result_R1)

f_result = eq1.rhs.evalf(subs={V1: 15@u_V, R1: 50@u_kOhm, R2: 10@u_kOhm})
print(f_result)

In [ ]:
d = schemdraw.Drawing(unit=1.4, inches_per_unit=0.4, lw=2, fontsize=9)

d += (op1 := elm.Opamp().at([5,0]).flip().label('$U_1$'))
d += elm.Line().left().at(op1.in1).length(d.unit).label('3', color='Grey')
d += elm.Dot()
d.push()
d += elm.Resistor().down().length(d.unit).label('R4\n10k')
d += elm.Ground()
d.pop()
d += elm.Line().up()
d += elm.Resistor().label('R3\n50k')
d += elm.Vdd()

d += elm.Line().left().length(d.unit*2).at(op1.in2).label('2', loc='start', color='Grey', ofst=(0.6, -0.5) )
d += elm.Dot()
d.push()
d += elm.Resistor().down().length(d.unit*2).label('R2\n100k')
d += elm.Ground()

d.pop()
d += elm.Diode().reverse().left().label('D1')
d += elm.Resistor().label('R1\n100k').length(d.unit*1.5).label('1', loc='start', color='Grey', ofst=(-0.1, -0.2) )
d += elm.Dot(open=True).label('IN', loc='left')

d += elm.Line().right().length(d.unit/2).at(op1.out).label('4', color='Grey')
d += elm.Dot()
d.push()
d += elm.Line().up()
d += elm.Diode().right().label('D2').label('5', loc='end', color='Grey', ofst=(-.2, .2) )
d += elm.ResistorVarIEEE().label('RV2\n1M')
d += elm.Line().down()

d.pop()
d += elm.Line().down()
d += elm.Diode().reverse().right().label('D3').label('6', loc='end', color='Grey', ofst=(-0.2, .2) )
d += elm.ResistorVarIEEE().label('RV2\n1M')
d += elm.Line().up()
d += elm.Dot()

d += elm.Line().right()
d += elm.Dot()
d.push()

d += elm.Line().right().label('7', color='Grey')
d += ( op2 := elm.Opamp().anchor('in2').label('$U_2$') )
      
d.pop()
d += elm.Capacitor().down().label('C1\n1u')
d += elm.Ground()

d += elm.Line().left().length(d.unit/2).at(op2.in1)
d += elm.Line().up()
d += elm.Line().right().length(d.unit*2.4)
d += elm.Line().down().toy(op2.out)
d += elm.Dot()
d.push()

d += elm.Line().left().tox(op2.out).label('8', color='Grey')
d.pop()
d += elm.Resistor().right().length(d.unit*2).label('R5\n1k')
d += elm.Dot(open=True).label('OUT', loc='right')

d.draw()

In [ ]:
ar_circuit = Circuit('ar')
ar_circuit.include(spice_library['TL072'])
ar_circuit.include(spice_library['D1N4148'])
ar_circuit.include(spice_library['CD4081B'])

ar_circuit.V('1', '+15V', ar_circuit.gnd, 'DC 15')
ar_circuit.V('2', '-15V', ar_circuit.gnd, 'DC -15')
V3 = ar_circuit.V('3', 'IN', ar_circuit.gnd, 'DC 0 AC 0 PULSE(0 5 20m 0m 0m 60m 200m)')

#the opamps
#                        NON-INVERTING INPUT
#                        |     INVERTING INPUT
#                        |     |      POSITIVE POWER SUPPLY
#                        |     |      |       NEGATIVE POWER SUPPLY
#                        |     |      |       |               OUTPUT
#                        |     |      |       |               |
ar_circuit.X(1, 'TL072', '2',  '3',   '+15V', ar_circuit.gnd, 'o')
ar_circuit.X(2, 'TL072', '7',  '8',   '+15V', ar_circuit.gnd, '8')

ar_circuit.R(1, 'IN', ar_circuit.gnd, 180@u_kΩ)
ar_circuit.R(3, '3', '+15V', 50@u_kΩ)
ar_circuit.R(4, '3', ar_circuit.gnd, 10@u_kΩ)
ar_circuit.R(5, '8', 'OUT', 1@u_kΩ)
ar_circuit.R(6, '4', 'o', 1@u_kΩ)

ar_circuit.D('1', 'IN', '2', model='D1N4148')
ar_circuit.D('2', '4', '5', model='D1N4148')
ar_circuit.D('3', '6', '4', model='D1N4148')

for c in ( VoltageDivider(R=1@u_MΩ, w=0.9, name='POT1'), 
        VoltageDivider(R=1@u_MΩ, w=0.99, name='POT2') ) :
    ar_circuit.subcircuit(c)
    
ar_circuit.X('RV1', 'POT1', '5', '5', '7')
ar_circuit.X('RV2', 'POT2', '6', '6', '7')
    
ar_circuit.C(1, '7', ar_circuit.gnd, 1@u_uF)

ar_simulator = ar_circuit.simulator(temperature=25, nominal_temperature=25)
ar_analysis = ar_simulator.transient(step_time=0.5@u_ms, start_time=0@u_ms, end_time=200@u_ms)


In [ ]:
from IPython.core.display import display, HTML
display(HTML('''
<section class="section">
	<div class="container">
		<div class="card is-fullwidth">
			<header class="card-header">
				<p class="card-header-title">spice netlist</p>
				<a class="card-header-icon card-toggle">
					<i class="fa fa-angle-down"></i>
				</a>
			</header>
			<div class="card-content is-hidden">
				<div class="content">
					<pre>%s</pre>
				</div>
			</div>
		</div>
	</div>
</section>''' % ar_circuit ))

In [ ]:
fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time [ms]')
ax1_buffer.set_ylabel('amplitude [V]')
ax1_buffer.plot(u_ms(ar_analysis['IN'].abscissa), ar_analysis['IN'], color='Grey')
#ax1_buffer.plot(u_ms(ar_analysis['4'].abscissa), ar_analysis['4'], color='Blue')
#ax1_buffer.plot(u_ms(ar_analysis['2'].abscissa), ar_analysis['2'], color='Green')
ax1_buffer.plot(u_ms(ar_analysis['OUT'].abscissa), ar_analysis['OUT'], color='Red')

ax1_buffer.legend(('trigger', 'envelope'), loc=(0.75,0.8))

plt.grid()
plt.tight_layout()
plt.show()

There is a problem with this circuit. The envelope will rise as long as the trigger is high and fall when it is low. The amplitude of the envelope is dependent on the trigger duty cycle. This is not very usefull. The circuit must be changed so that it creates a constant amplitude.

In [ ]:
d = schemdraw.Drawing(unit=1.4, inches_per_unit=0.4, lw=2, fontsize=9)

d += (op1 := elm.Opamp().at([5,0]).flip().label('$U_1$', ofst=(0, -.2)))
d += elm.Line().left().at(op1.in1).length(d.unit/2)
d += elm.Ground()

d += elm.Line().left().length(d.unit).at(op1.in2).label('2', loc='start', color='Grey', ofst=(0.3, -0.4) )
d += ( D4 := elm.Dot())
d.push()

d += elm.Resistor().label('R2\n27k')
d += elm.Diode().reverse().left().label('D1').label('1', loc='start', color='Grey', ofst=(-0, -0.2) )
d += elm.Dot()
d.push()
d += elm.Line().length(d.unit/2)
d += elm.Dot(open=True).label('IN', loc='left')
d.pop()
d += elm.Resistor().down().label('R1\n100k')
d += elm.Ground()

d.pop()
d += elm.Line().up()
d += elm.Resistor().right().length(d.unit*3).label('R3\n470k')
d += elm.Line().down().toy(op1.out)
d += elm.Dot()
d.push()

d += elm.Line().right().length(d.unit/2).at(op1.out).label('3', color='Grey')

d.pop()
d += elm.Diode().down().label('D2').label('4', loc='end', color='Grey', ofst=(-0.2, .2))
d += elm.Diode().down().label('D3').label('5', loc='end', color='Grey', ofst=(-0.2, .2))
d += elm.Dot()
d.push()
d += elm.Resistor().label('R4\n2.2k')
d += ( D3 := elm.Dot().label('6', color='Grey', ofst=(-0.3, -0.3))) 
d += elm.Resistor().label('R5\n47k')
d += elm.Ground()

d.pop()
d += elm.Line().right()
d += ( op2 := elm.Opamp().anchor('in2').label('$U_2$', ofst=(0, -.2)) )

d += elm.Line().left().length(d.unit/2).at(op2.in1).label('7', color='Grey', ofst=(0, -.2))
d += elm.Line().up()
d += elm.Dot()
d.push()
d += elm.Diode().right().length(d.unit*2.5).label('D5')
d += elm.Line().down().toy(op2.out)
d += (D1 := elm.Dot())
d += elm.Line().left().tox(op2.out)
d.pop()
d += elm.Line().up()
d += (D5 := elm.Dot())
d += elm.Diode().right().reverse().length(d.unit*2.5).label('D4')
d += elm.Line().down()
d += elm.Dot()

d += elm.Line().right().at(D1.end).label('8', color='Grey', ofst=(0, .2))
d += elm.Dot()
d.push()
d += elm.Line().up()
d += elm.Diode().right().label('D6').label('10', loc='end', color='Grey', ofst=(-0.3, .2))
d += elm.ResistorVarIEEE().label('RV2\n1M')
d += elm.Line().down()

d.pop()
d += elm.Line().down()
d += elm.Diode().reverse().right().label('D7').label('9', loc='end', color='Grey', ofst=(-0.2, .2) )
d += elm.ResistorVarIEEE().label('RV2\n1M')
d += elm.Line().up()
d += elm.Dot()

d += elm.Line().right()
d += elm.Dot()
d.push()

d += elm.Line().right().label('11', color='Grey')
d += ( op3 := elm.Opamp().anchor('in2').label('$U_3$', ofst=(0, -.2)))
      
d.pop()
d += elm.Capacitor().down().label('C1\n1u')
d += elm.Ground()

d += elm.Line().left().length(d.unit/2).at(op3.in1)
d += elm.Line().up()
d += elm.Line().right().length(d.unit*2.5)
d += ( Dout := elm.Dot())
d += elm.Line().down().toy(op3.out)
d += ( D2 := elm.Dot())
d += elm.Line().left().tox(op3.out).label('12', color='Grey')
d += elm.Resistor().right().length(d.unit*2).label('R7\n1k').at(D2.end)
d += elm.Dot(open=True).label('OUT', loc='right')

d += elm.Line().down().length(d.unit*4.5).at(D2.end)
d += elm.Line().left().length(d.unit*6)
d += ( op4 := elm.Opamp().anchor('in1').label('$U_4$', ofst=(0, -.2)))

d += elm.Line().right().length(d.unit/2).at(op4.in2)
d += elm.Line().up().toy(D3.end)
d += elm.Line().left().tox(D3.end)

d += elm.Diode().reverse().left().at(op4.out).tox(D4.end).label('D8') \
    .label('13', loc='start', color='Grey', ofst=(1, -0.3)) \
    .label('14', loc='end', color='Grey', ofst=(-1, -0.3))
d += elm.Resistor().up().toy(D4.end).label('R8\n390k')

d += elm.Line().up().length(d.unit*1.5).at(Dout.end)
d += elm.Resistor().left().tox(D5.end).label('R6\n10k')
d += elm.Line().down().toy(D5.end)

d.draw()

In [ ]:
V1, V2, R1, R2 = sp.symbols("V1 V2 R1 R2")
eq1 = sp.Eq(V2, V1 * R2 / (R1 + R2))

eq1_R1 = sp.solve(eq1, R1)[0]
print(eq1_R1)
f_result_R1 = eq1_R1.evalf(subs={R2: 10@u_kOhm, V1: 15@u_V, V2: 2.5@u_V})
print(f_result_R1)

f_result = eq1.rhs.evalf(subs={V1: 15@u_V, R1: 2.2@u_kOhm, R2: 47@u_kOhm})
print(f_result)


In [ ]:
ar_circuit = Circuit('ad_ar')
ar_circuit.include(spice_library['TL072'])
ar_circuit.include(spice_library['D1N4148'])
ar_circuit.include(spice_library['CD4081B'])

ar_circuit.V('1', '+15V', ar_circuit.gnd, 'DC 15')
ar_circuit.V('2', '-15V', ar_circuit.gnd, 'DC -15')
V3 = ar_circuit.V('3', 'IN', ar_circuit.gnd, 'DC 0 AC 0 PULSE(0 5 20m 0m 0m 10m 200m)')

#the opamps
#                     NON-INVERTING INPUT
#                     |           INVERTING INPUT
#                     |           |             POSITIVE POWER SUPPLY
#                     |           |             |       NEGATIVE POWER SUPPLY
#                     |           |             |       |       OUTPUT
#                     |           |             |       |       |
ar_circuit.X(1, 'TL072', '2',        ar_circuit.gnd,          '+15V', '-15V', '3')
ar_circuit.X(2, 'TL072', '5',        '7',          '+15V', '-15V', '8')
ar_circuit.X(3, 'TL072', '11',       '12',          '+15V', '-15V', '12')
ar_circuit.X(4, 'TL072', '6',        '12',        '+15V', '-15V', '13')

#CV voltage divider
ar_circuit.R(1, 'IN', ar_circuit.gnd, 100@u_kΩ)
ar_circuit.R(2, '1', '2', 27@u_kΩ)
ar_circuit.R(3, '2', '3', 470@u_kΩ)
ar_circuit.R(4, '5', '6', 2.2@u_kΩ)
ar_circuit.R(5, '6', ar_circuit.gnd, 47@u_kΩ)
ar_circuit.R(6, '12', '7', 10@u_kΩ)
ar_circuit.R(7, '12', 'OUT', 1@u_kΩ)
ar_circuit.R(8, '14', '2', 390@u_kΩ)

ar_circuit.D('1', 'IN', '1', model='D1N4148')
ar_circuit.D('2', '3', '4', model='D1N4148')
ar_circuit.D('3', '4', '5', model='D1N4148')
ar_circuit.D('4', '8', '7', model='D1N4148')
ar_circuit.D('5', '7', '8', model='D1N4148')
ar_circuit.D('6', '8', '10', model='D1N4148')
ar_circuit.D('7', '9', '8', model='D1N4148')
ar_circuit.D('8', '14', '13', model='D1N4148')

for c in ( VoltageDivider(R=1@u_MΩ, w=0.99, name='POT1'), 
        VoltageDivider(R=1@u_MΩ, w=0.99, name='POT2') ) :
    ar_circuit.subcircuit(c)
    
ar_circuit.X('RV1', 'POT1', '10', '10', '11')
ar_circuit.X('RV2', 'POT2', '9', '9', '11')
    
ar_circuit.C(1, '11', ar_circuit.gnd, 2.2@u_uF)

ar_simulator = ar_circuit.simulator(temperature=25, nominal_temperature=25)
ar_analysis = ar_simulator.transient(step_time=1@u_ms, start_time=0@u_ms, end_time=400@u_ms)


In [ ]:
from IPython.core.display import display, HTML
display(HTML('''
<section class="section">
	<div class="container">
		<div class="card is-fullwidth">
			<header class="card-header">
				<p class="card-header-title">spice netlist</p>
				<a class="card-header-icon card-toggle">
					<i class="fa fa-angle-down"></i>
				</a>
			</header>
			<div class="card-content is-hidden">
				<div class="content">
					<pre>%s</pre>
				</div>
			</div>
		</div>
	</div>
</section>''' % ar_circuit ))

In [ ]:
fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time [ms]')
ax1_buffer.set_ylabel('amplitude [V]')
ax1_buffer.plot(u_ms(ar_analysis['IN'].abscissa), ar_analysis['IN'], color='Grey')
ax1_buffer.plot(u_ms(ar_analysis['6'].abscissa), ar_analysis['6'], color='Blue')
ax1_buffer.plot(u_ms(ar_analysis['OUT'].abscissa), ar_analysis['OUT'], color='Red')

ax1_buffer.legend(('IN', 'Node 6', 'OUT'), loc=(0.75,0.8))

plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
V3.dc_value = "PULSE(0 5 20m 0m 0m 200m 400m)"
ar_simulator = ar_circuit.simulator(temperature=25, nominal_temperature=25)
ar_analysis = ar_simulator.transient(step_time=1@u_ms, start_time=0@u_ms, end_time=400@u_ms)


fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time [ms]')
ax1_buffer.set_ylabel('amplitude [V]')
ax1_buffer.plot(u_ms(ar_analysis['IN'].abscissa), ar_analysis['IN'], color='Grey')
ax1_buffer.plot(u_ms(ar_analysis['6'].abscissa), ar_analysis['6'], color='Blue')
ax1_buffer.plot(u_ms(ar_analysis['OUT'].abscissa), ar_analysis['OUT'], color='Red')

ax1_buffer.legend(('IN', 'Node 6', 'OUT'), loc=(0.75,0.8))

plt.grid()
plt.tight_layout()
plt.show()

# end of envelope

In [ ]:
V1, V2, R1, R2 = sp.symbols("V1 V2 R1 R2")
eq1 = sp.Eq(V2, V1 * R2 / (R1 + R2))

eq1_R1 = sp.solve(eq1, R1)[0]
print(eq1_R1)
f_result_R1 = eq1_R1.evalf(subs={R2: 1@u_kOhm, V1: 15@u_V, V2: 0.4@u_V})
print(f_result_R1)

f_result = eq1.rhs.evalf(subs={V1: 15@u_V, R1: 50@u_kOhm, R2: 10@u_kOhm})
print(f_result)

U, R, I = sp.symbols("U R I")
eq1 = sp.Eq(U, R * I)
eq1_I = sp.solve(eq1, I)[0]
i_result = eq1_I.evalf(subs={U: 15@u_V, R: 10@u_kOhm})
print(u_mA(i_result*1000))

eq1_R = sp.solve(eq1, R)[0]
r_result = eq1_R.evalf(subs={U: 15@u_V, I: i_result/100})
print(u_kOhm(r_result))


In [ ]:
d = schemdraw.Drawing(unit=1.4, inches_per_unit=0.4, lw=2, fontsize=9)

d += (op1 := elm.Opamp().at([5,0]).flip().label('$U_1$', ofst=(0, -.2)))
d += elm.Line().left().at(op1.in1).length(d.unit/2)
d += elm.Ground()

d += elm.Line().left().length(d.unit).at(op1.in2).label('2', loc='start', color='Grey', ofst=(0.3, -0.4) )
d += ( D4 := elm.Dot())
d.push()

d += elm.Resistor().label('R2\n27k')
d += elm.Diode().reverse().left().label('D1').label('1', loc='start', color='Grey', ofst=(-0, -0.2) )
d += elm.Dot()
d.push()
d += elm.Line().length(d.unit/2)
d += elm.Dot(open=True).label('IN', loc='left')
d.pop()
d += elm.Resistor().down().label('R1\n100k')
d += elm.Ground()

d.pop()
d += elm.Line().up()
d += elm.Resistor().right().length(d.unit*3).label('R3\n470k')
d += elm.Line().down().toy(op1.out)
d += elm.Dot()
d.push()

d += elm.Line().right().length(d.unit/2).at(op1.out).label('3', color='Grey')

d.pop()
d += elm.Diode().down().label('D2').label('4', loc='end', color='Grey', ofst=(-0.2, .2))
d += elm.Diode().down().label('D3').label('5', loc='end', color='Grey', ofst=(-0.2, .2))
d += elm.Dot()
d.push()
d += elm.Resistor().length(d.unit*1.5).label('R4\n2.2k')
d += ( D3 := elm.Dot().label('6', color='Grey', ofst=(-0.3, -0.3))) 
d += elm.Resistor().length(d.unit*1.0).label('R5\n47k')
d += elm.Ground()

d.pop()
d += elm.Line().right()
d += ( op2 := elm.Opamp().anchor('in2').label('$U_2$', ofst=(0, -.2)) )

d += elm.Line().left().length(d.unit/2).at(op2.in1).label('7', color='Grey', ofst=(0, -.2))
d += elm.Line().up()
d += elm.Dot()
d.push()
d += elm.Diode().right().length(d.unit*2.5).label('D5')
d += elm.Line().down().toy(op2.out)
d += (D1 := elm.Dot())
d += elm.Line().left().tox(op2.out)
d.pop()
d += elm.Line().up()
d += (D5 := elm.Dot())
d += elm.Diode().right().reverse().length(d.unit*2.5).label('D4')
d += elm.Line().down()
d += elm.Dot()

d += elm.Line().right().at(D1.end).label('8', color='Grey', ofst=(0, .2))
d += elm.Dot()
d.push()
d += elm.Line().up()
d += elm.Diode().right().label('D6').label('10', loc='end', color='Grey', ofst=(-0.3, .2))
d += elm.ResistorVarIEEE().label('RV2\n1M')
d += elm.Line().down()

d.pop()
d += elm.Line().down()
d += elm.Diode().reverse().right().label('D7').label('9', loc='end', color='Grey', ofst=(-0.2, .2) )
d += elm.ResistorVarIEEE().label('RV2\n1M')
d += elm.Line().up()
d += elm.Dot()

d += elm.Line().right()
d += elm.Dot()
d.push()

d += elm.Line().right().label('11', color='Grey')
d += ( op3 := elm.Opamp().anchor('in2').label('$U_3$', ofst=(0, -.2)))
      
d.pop()
d += elm.Capacitor().down().label('C1\n1u')
d += elm.Ground()

d += elm.Line().left().length(d.unit/2).at(op3.in1)
d += elm.Line().up()
d += elm.Line().right().length(d.unit*2.5)
d += ( Dout := elm.Dot())
d += elm.Line().down().toy(op3.out)
d += ( D2 := elm.Dot())
d += elm.Line().left().tox(op3.out).label('12', color='Grey')
d += elm.Resistor().right().length(d.unit*2).label('R7\n1k').at(D2.end)
d += elm.Dot(open=True).label('OUT', loc='right')

d += elm.Line().down().length(d.unit*5).at(D2.end)
d += elm.Line().left().length(d.unit*6)
d += ( op4 := elm.Opamp().anchor('in1').label('$U_4$', ofst=(0, -.2)))

d += elm.Line().right().length(d.unit/2).at(op4.in2)
#d += elm.Dot()
#d.push()
d += elm.Line().up().toy(D3.end)
d += elm.Dot()
d += elm.Line().left().tox(D3.end)

d += elm.Diode().reverse().left().at(op4.out).tox(D4.end).label('D8') \
    .label('13', loc='start', color='Grey', ofst=(1, -0.3)) \
    .label('14', loc='end', color='Grey', ofst=(-1, -0.3))
d += elm.Resistor().up().toy(D4.end).label('R8\n390k')

d += elm.Line().up().length(d.unit*1.5).at(Dout.end)
d += elm.Resistor().left().tox(D5.end).label('R6\n10k')
d += elm.Line().down().toy(D5.end)

#d.pop()
#d += elm.Line().down().length(d.unit*6)
#d += elm.Line().right().length(d.unit*2)
d += ( U6 := elm.Opamp().right().at([17,-15.5]).label('$U_6$', ofst=(0, -.2)))
#d += elm.Line().left().length(d.unit/2).at(U6.in1)
#d += elm.Ground()
d += elm.elm.Diode().right().at(U6.out).label('18', loc='start', color='Grey', ofst=(0.3, .3)).label('D10')
d += elm.Line().up().length(d.unit*1.5).label('19', loc='end', color='Grey', ofst=(-0.6, .6))
d += elm.Dot()
d += elm.Resistor().right().length(d.unit*2).label('R12\n100k') \
    .label('20', loc='end', color='Grey', ofst=(-0.6, .3))
d += elm.Dot()
d.push()
d += elm.Diode().down().reverse().label('D11')
d += elm.Ground()

d.pop()
d += ( U7 := logic.Nor().anchor('in2').label('$U_7$'))
#d += elm.Line().left().at(U7.in1)
#d += elm.Line().up()
d += elm.Line().right().at(U7.out).length(d.unit/2)
d += elm.Dot(open=True).label('EOE', loc='right')

d += ( U5 := elm.Opamp().at((U6.in2.x, U6.in2.y+d.unit*3)).label('$U_5$', ofst=(0, -.2)))

d += elm.Line().left().length(d.unit/2).at(U5.in1)
d += elm.Dot()
d.push()
d += elm.Line().up().length(d.unit/4)
d += elm.Resistor().up().label('R9\n36k')
d += elm.Vdd()
d.pop()
d += elm.Resistor().down().label('R10\n1k').toy(U6.in2) \
    .label('15', loc='start', color='Grey', ofst=(0.5, -.3)) \
    .label('16', loc='end', color='Grey', ofst=(-0.5, -.6))
d += elm.Dot()
d.push()
d += elm.Line().right().tox(U6.in2)

d.pop()
d += elm.Resistor().down().length(d.unit*1.5).label('R11\n220')
d += elm.Ground()
d += elm.Diode().right().at(U5.out).label('17', loc='start', color='Grey', ofst=(0.3, .3)).label('D9')
d += elm.Line().down().length(d.unit*1.5)

d += elm.Line().left().at(U5.in2).length(d.unit*1.5)
d += elm.Line().up().toy(op4.in1)
d += elm.Dot()

d += elm.Line().left().at(U6.in1).length(d.unit*1.5)
d += elm.Line().up().toy(U5.in2)
d += elm.Dot()

d += elm.Line().up().at(U7.in1).toy(D3.end)
d += elm.Line().left().tox(D3.end)

d.draw()

In [ ]:
ar_circuit = Circuit('ad_ar_eoe')
ar_circuit.include(spice_library['TL072'])
ar_circuit.include(spice_library['D1N4148'])
ar_circuit.include(spice_library['CD4081B'])
ar_circuit.include(spice_library['BC846B'])
ar_circuit.include(spice_library['CD4002B'])

ar_circuit.V('1', '+15V', ar_circuit.gnd, 'DC 15')
ar_circuit.V('2', '-15V', ar_circuit.gnd, 'DC -15')
V3 = ar_circuit.V('3', 'IN', ar_circuit.gnd, 'DC 0 AC 0 PULSE(0 5 20m 0m 0m 10m)')
#V3 = ar_circuit.V('3', 'IN', ar_circuit.gnd, 'DC 0 AC 0 PULSE(0 5 20m 0m 0m 10m 150m)')

#the opamps
#                     NON-INVERTING INPUT
#                     |           INVERTING INPUT
#                     |           |             POSITIVE POWER SUPPLY
#                     |           |             |       NEGATIVE POWER SUPPLY
#                     |           |             |       |       OUTPUT
#                     |           |             |       |       |
ar_circuit.X(1, 'TL072', '2',        ar_circuit.gnd,          '+15V', '-15V', '3')
ar_circuit.X(2, 'TL072', '5',        '7',          '+15V', '-15V', '8')
ar_circuit.X(3, 'TL072', '11',       '12',          '+15V', '-15V', '12')
ar_circuit.X(4, 'TL072', '6',        '12',        '+15V', '-15V', '13')

ar_circuit.X(5, 'TL072', '12',       '15',        '+15V', '-15V', '17')
ar_circuit.X(6, 'TL072', '16',       '12',     '+15V', '-15V', '18')
ar_circuit.X(7, 'CD4002B', 'EOE', '6',  '20',   '+15V', ar_circuit.gnd)

#ar_circuit.BJT(7, '24', '22', ar_circuit.gnd, model="BC846B")
#ar_circuit.BJT(8, '24', '23', ar_circuit.gnd, model="BC846B")

#ar_circuit.R(20, '+15V', '20', 36.5@u_kΩ)
#ar_circuit.R(21, '20', '20a', 1@u_kΩ)
#ar_circuit.R(25, '20a', ar_circuit.gnd, 400@u_Ω)
#ar_circuit.R(22, '21', '22', 100@u_kΩ)
#ar_circuit.R(23, '6', '23', 100@u_kΩ)
#ar_circuit.R(24, '+15V', '24', 10@u_kΩ)
#ar_circuit.D('20', '21a', '21', model='D1N4148')
#ar_circuit.D('21', '22a', '21', model='D1N4148')

#CV voltage divider
ar_circuit.R(1, 'IN', ar_circuit.gnd, 100@u_kΩ)
ar_circuit.R(2, '1', '2', 27@u_kΩ)
ar_circuit.R(3, '2', '3', 470@u_kΩ)
ar_circuit.R(4, '5', '6', 2.2@u_kΩ)
ar_circuit.R(5, '6', ar_circuit.gnd, 47@u_kΩ)
ar_circuit.R(6, '12', '7', 10@u_kΩ)
ar_circuit.R(7, '12', 'OUT', 1@u_kΩ)
ar_circuit.R(8, '14', '2', 390@u_kΩ)
R9 = ar_circuit.R(9, '+15V', '15', 36@u_kΩ)
R10 = ar_circuit.R(10, '15', '16', 1@u_kΩ)
R11 = ar_circuit.R(11, '16', ar_circuit.gnd, 220@u_Ω)
ar_circuit.R(12, '19', '20', 100@u_kΩ)

ar_circuit.D('1', 'IN', '1', model='D1N4148')
ar_circuit.D('2', '3', '4', model='D1N4148')
ar_circuit.D('3', '4', '5', model='D1N4148')
ar_circuit.D('4', '8', '7', model='D1N4148')
ar_circuit.D('5', '7', '8', model='D1N4148')
ar_circuit.D('6', '8', '10', model='D1N4148')
ar_circuit.D('7', '9', '8', model='D1N4148')
ar_circuit.D('8', '14', '13', model='D1N4148')
ar_circuit.D('9', '17', '19', model='D1N4148')
ar_circuit.D('10', '18', '19', model='D1N4148')
ar_circuit.D('11', ar_circuit.gnd, '20', model='D1N4148')

for c in ( VoltageDivider(R=1@u_MΩ, w=0.99, name='POT1'), 
        VoltageDivider(R=1@u_MΩ, w=0.99, name='POT2') ) :
    ar_circuit.subcircuit(c)
    
ar_circuit.X('RV1', 'POT1', '10', '10', '11')
ar_circuit.X('RV2', 'POT2', '9', '9', '11')
    
ar_circuit.C(1, '11', ar_circuit.gnd, 2.2@u_uF)

ar_simulator = ar_circuit.simulator(temperature=25, nominal_temperature=25)
ar_analysis = ar_simulator.transient(step_time=1@u_ms, start_time=0@u_ms, end_time=400@u_ms)
ar_op_analysis = ar_simulator.operating_point()

print("Voltage Divider: %s, %s " % (float(ar_op_analysis['15']), float(ar_op_analysis['16'])))

In [ ]:
from IPython.core.display import display, HTML
display(HTML('''
<section class="section">
	<div class="container">
		<div class="card is-fullwidth">
			<header class="card-header">
				<p class="card-header-title">spice netlist</p>
				<a class="card-header-icon card-toggle">
					<i class="fa fa-angle-down"></i>
				</a>
			</header>
			<div class="card-content is-hidden">
				<div class="content">
					<pre>%s</pre>
				</div>
			</div>
		</div>
	</div>
</section>''' % ar_circuit ))

In [ ]:
fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time [ms]')
ax1_buffer.set_ylabel('amplitude [V]')
#ax1_buffer.plot(u_ms(ar_analysis['21'].abscissa[530:850]), ar_analysis['21'][530:850], color='Grey')
#ax1_buffer.plot(u_ms(ar_analysis['22'].abscissa[530:850]), ar_analysis['22'][530:850], color='Yellow')
#ax1_buffer.plot(u_ms(ar_analysis['14'].abscissa), ar_analysis['14'], color='Blue')

#ax1_buffer.plot(u_ms(ar_analysis['6'].abscissa), (ar_analysis['6']), color='Blue')


ax1_buffer.plot(u_ms(ar_analysis['19'].abscissa), (ar_analysis['19']), color='Green')
ax1_buffer.plot(u_ms(ar_analysis['OUT'].abscissa), (ar_analysis['OUT']), color='Orange')
ax1_buffer.plot(u_ms(ar_analysis['EOE'].abscissa), ar_analysis['EOE'], color='Red')

ax1_buffer.legend(('Node 21', 'Node 22', 'EOE', 'envelope'), loc=(0.75,0.7))

plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
U, R, I = sp.symbols("U R I")
eq1 = sp.Eq(U, R * I)
eq1_I = sp.solve(eq1, I)[0]
i_result = eq1_I.evalf(subs={U: 15@u_V, R: R9.resistance + R10.resistance + R11.resistance})
print(u_mA(i_result*1000))

v1_result = eq1.evalf(subs={R: R9.resistance, I: i_result})
v2_result = eq1.evalf(subs={R: R10.resistance, I: i_result})

print("Voltage Divider: %s, %s " % (15. - v1_result.rhs, 15 - v1_result.rhs - v2_result.rhs))

For the end of envelope trigger we use another comperator. This comperator will trigger at about 0.6V. The trigger starts on he falling edge and ends on the rising edge at 0.6 V again (Orange). An AND gate will limit the trigger to the falling gate. One input is the output of the comperator the other the inverted output of the latch.

In [ ]:
from IPython.core.display import display, HTML
display(HTML('''
<script>
document.addEventListener('DOMContentLoaded', function() {
	let cardToggles = document.getElementsByClassName('card-toggle');
	for (let i = 0; i < cardToggles.length; i++) {
		cardToggles[i].addEventListener('click', e => {
			e.currentTarget.parentElement.parentElement.childNodes[3].classList.toggle('is-hidden');
		});
	}
});
</script>
'''))

https://studylib.net/doc/18188991/the-all-in-ad-ar-envelope-generator 
https://electro-music.com/forum/phpbb-files/fritzdualadar_builddoc_283.pdf 
https://nanopdf.com/download/the-all-in-ad-ar-envelope-generator_pdf
https://www.renesas.com/us/en/document/dst/icm7555-icm7556-datasheet


https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgpABZsKBTAWjDACgAnEYw8MFbrxp4oIbIXidB4GjWkoaVKmEkAlacOlheS2sqS6YCNgHN5i+XhFK2Adxlz+InuBTW7DkAqovvUD2Cy4BIgTNgC2jb2gXJoIjHBUVpBLk7+9qluWvz+ACZeFgm4jkECuQwAZgCGAK4ANgAubPl+KApikLHtZZW1jR7FnoNtcpADnUPinmPqYREhTISOIVT0FgaiRs16VH6KyqUg5dX1TQAOtJppCEFpEDZm+wVUT4M2XNhTCU8JyvBwbAaYkIIk0NG0tGs4Cg0AQ-BQuGIeEggRwkCWMIQGHBVmwigwYCsCAIyCoxz6TSBPyCsniQWUmJoKBUGFIxOE6OJMIkikICGwCCWYB4YAwvBUZN6p1MtAhGPBvGRmxltMSl3iYuVXAVYgEOrmoglAPsOvC6t16XNZtVaRmHTkmj2UL1VA2SlhbHU+utEKYml0hTdmKkg004lBUKNAKqngSfgS8SQLCQUbgMMUN0gmezGbooRosOICFIJaRGGJRdJkhjce6FhGIEToTAKf+ab9heLpbLFY2qbYAHtgo3RIokV53fdqIFDXJ6ERRAvsIPwPQoWORGgYRAIAIdGwgA